In [3]:
# LIBRARY
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import re
import time
import os
# END LIBRARY

In [16]:
# Function for Open Chrome and Login to LinkedIN
def open_chrome():
    chrome_options = Options()
    chrome_options.add_argument("--start-maximized")  # Buka browser dalam mode layar penuh

    driver = webdriver.Chrome(options=chrome_options)

    return driver
# End Function for Open Chrome and Login to LinkedIN

def import_to_csv(company_names,job_titles,posted_ons,job_details,image_src,type_location,locations,job,z):
    print("Masuk Sini")
    df = pd.DataFrame({
        "Company Names":company_names, "Job Titles":job_titles,"Posted On":posted_ons,"Job Details":job_details,
        "Images":image_src,"Type Location":type_location,"Locations":locations}) 
    df = df.drop_duplicates()
    df.to_csv(job+"_"+z+".csv", index=False) # Nama Ubah Sesuai Search

company_names = []
job_titles = []
posted_ons = []
job_details = []
image_src = []
type_location = []
locations = []

# Function for Scrapping data job in LinkedIn
def find_jobs(driver) : 
    jobs = ['IoT','Generative AI','Software Engineer','Product Manager']
    
    for job in jobs:
        # try :
        # Buka LinkedIn
        driver.get("https://id.indeed.com/")

        job_search = driver.find_element(By.XPATH, "//input[@id='text-input-what']")
        time.sleep(3)
        job_search.send_keys(job)
        job_search.send_keys(Keys.RETURN)

        company_names = []
        job_titles = []
        posted_ons = []
        job_details = []
        image_src = []
        type_location = []
        locations = []
        link_job = []
        time.sleep(5)
        driver.execute_script("window.scrollBy(0, 700);")
        for z in range(30):

            # Temukan semua elemen <li> di dalam <ul> tersebut
            job_title_find = driver.find_elements(By.XPATH, "//a[@class='jcs-JobTitle css-jspxzf eu4oa1w0']")
            job_ids = [li.get_attribute("id") for li in job_title_find if li.get_attribute("id") is not None]
            for id in job_ids :
                teks = driver.find_element(By.XPATH, "//a[@id='"+id+"']")
                print("LINK : ",teks.get_attribute('href'))
                link_job.append(teks.get_attribute('href'))
                job_titles.append(teks.text)

            # Temukan semua elemen <li> di dalam <ul> tersebut
            company_find = driver.find_elements(By.XPATH, "//span[@class='css-63koeb eu4oa1w0']")
            for id in company_find :
                company_names.append(id.text)
            
            print(company_names)
            # Temukan semua elemen <li> di dalam <ul> tersebut
            li_elements = driver.find_elements(By.XPATH, "//span[@class='css-qvloho eu4oa1w0']")
            for id in li_elements :
                try :
                    posted_ons.append(id.text.split("\n")[1])
                except Exception as e :
                    posted_ons.append("")    

            location_find = driver.find_elements(By.XPATH, "//div[@class='css-1p0sjhy eu4oa1w0']")
            for id in location_find :
                locations.append(id.text)

            print("Banyak jobs ID : ", len(job_ids))
            for id in job_ids :
                job_click = driver.find_element(By.XPATH, "//a[@id='"+id+"']")
                job_click.click()
                time.sleep(2)
                # WebDriverWait setup, timeout dalam 10 detik
                wait = WebDriverWait(driver, 10)

                # Tunggu hingga elemen dapat ditemukan dan terlihat
                div_details = wait.until(EC.visibility_of_element_located((By.XPATH, "//div[@class='jobsearch-JobComponent css-17riagq eu4oa1w0']")))
                try :
                    div_select = div_details.find_element(By.XPATH, "//div[@class='js-match-insights-provider-16m282m e37uo190' and contains(., 'Job type')]")
                    type_location_find = div_select.find_elements(By.XPATH, "//li[@class='js-match-insights-provider-hj3618 eu4oa1w0']")
                    sementara = []        
                    for types in type_location_find:
                        sementara.append(types.text)
                    
                    # Menghapus elemen yang mengandung angka
                    cleaned_list = [item for item in sementara if not re.search(r'\d', item)]
                    print(",".join(cleaned_list))
                    type_location.append(",".join(cleaned_list))
                except Exception as e :
                    type_location.append("")
                
                time.sleep(2)
                
                # Temukan elemen div dengan class yang dimaksud
                element = driver.find_element(By.XPATH, "//div[@class='jobsearch-embeddedBody css-vzhwvw eu4oa1w0']")

                # Lakukan scroll ke bagian paling bawah
                driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
                time.sleep(1)
                detail_find = div_details.find_element(By.XPATH, "//div[@id='jobDescriptionText']")
                job_details.append(detail_find.text)
                
                try:
                    img_find = div_details.find_element(By.XPATH, "//div[@class='jobsearch-JobInfoHeader-title-container jobsearch-JobInfoHeader-title-containerEji css-82ysm6 eu4oa1w0']")
                    img_element = img_find.find_element(By.TAG_NAME, "img")
                    img_element = img_element.get_attribute("src")
                    image_src.append(img_element)
                except Exception as e :
                    image_src.append("")

                print(len(company_names))
                print(len(job_titles))
                print(len(job_details))
                print(len(posted_ons))
                print(len(image_src))
                print(len(type_location))
                print(len(locations))

            path_loc = "D:\Codelabs\Kompetisi\AIC Compfest\Projek\Indeed"
            # Gabungkan path folder dengan nama file
            file_path = os.path.join(path_loc, f"{job}_{z}.csv")
            
            df = pd.DataFrame({
                "Company Names":company_names, "Job Titles":job_titles,"Posted On":posted_ons,"Job Details":job_details,
                "Images":image_src,"Type Location":type_location,"Locations":locations, "Link": link_job}) 
            
            df.to_csv(file_path, index=False)
            try :
                
                next = driver.find_element(By.XPATH, "//a[@aria-label='Next Page']")
                next.click()
                time.sleep(2)
                try:
                    time.sleep(3)
                    # import_to_csv(company_names,job_titles,posted_ons,job_details,skills,job)
                    
                    print("MASUK SINI")
                    close = driver.find_element(By.XPATH, "//button[@aria-label='close']")
                    print(close)
                    close.click()
                except Exception as e :
                    print()
                driver.execute_script("window.scrollBy(0, 700);")
            except Exception as e :
                break


In [17]:
driver = open_chrome()
find_jobs(driver)

LINK :  https://id.indeed.com/rc/clk?jk=85fa710d80adb011&bb=O4EpJZmqyM4Hty7KUxju62ecMQIldHBfawaoGcx0Ti_mnA__eiz3AN2Aoo38b2U9Ph5YI70AZTTCyH17vH365n_v410ZkJtS3nSNyA3xUjkTOrkMCjXYlkgdW4MtMsez&xkcb=SoBt67M38fkbtdQ82R0LbzkdCdPP&fccid=055cccd4b5b64775&vjs=3
LINK :  https://id.indeed.com/rc/clk?jk=eb60f338c705c1ea&bb=O4EpJZmqyM4Hty7KUxju61thXrOqYXyDN7oWQQ2rJmTYRctuW-T_dvAQkZKoOxz2-hCJFBOwLGsUN08nzuglMpd4KkZd3WoA_cjdNDBjTLASZ4PVrRIGJkzegLgNKHsQ&xkcb=SoDZ67M38fkbtdQ82R0KbzkdCdPP&fccid=a2faf1301ac6ad4b&vjs=3
LINK :  https://id.indeed.com/rc/clk?jk=a9015afb2984a76f&bb=O4EpJZmqyM4Hty7KUxju665dp6RPELAFM4JIi_l0Y2ia5LrudKTnHLHSYLEWmI1xDg_4nf0F7y6KPHsqlHRa3SLLIcYrU8guXoC4Gf_aE-xFHUHJ3MPAybxaBSoIp2mH&xkcb=SoBE67M38fkbtdQ82R0JbzkdCdPP&fccid=44981632e7243e42&vjs=3
LINK :  https://id.indeed.com/rc/clk?jk=d03caeb9d72d8932&bb=O4EpJZmqyM4Hty7KUxju6xX-aFCdZu4Lv0qRgDhdb16jmzxDQuTpmMkceF-fxmg4XaMjLpyK9DLeUyCG7PToTTTWV1uOWCh59J4wWlCos6Xyw0rXQnWHQ2Eii-eU9OfG&xkcb=SoDw67M38fkbtdQ82R0IbzkdCdPP&fccid=e8ccf05808b5bef

TimeoutException: Message: 
Stacktrace:
	GetHandleVerifier [0x00007FF74753EEB2+31554]
	(No symbol) [0x00007FF7474B7EE9]
	(No symbol) [0x00007FF74737872A]
	(No symbol) [0x00007FF7473C8434]
	(No symbol) [0x00007FF7473C853C]
	(No symbol) [0x00007FF74740F6A7]
	(No symbol) [0x00007FF7473ED06F]
	(No symbol) [0x00007FF74740C977]
	(No symbol) [0x00007FF7473ECDD3]
	(No symbol) [0x00007FF7473BA33B]
	(No symbol) [0x00007FF7473BAED1]
	GetHandleVerifier [0x00007FF747848B2D+3217341]
	GetHandleVerifier [0x00007FF747895AF3+3532675]
	GetHandleVerifier [0x00007FF74788B0F0+3489152]
	GetHandleVerifier [0x00007FF7475EE786+750614]
	(No symbol) [0x00007FF7474C376F]
	(No symbol) [0x00007FF7474BEB24]
	(No symbol) [0x00007FF7474BECB2]
	(No symbol) [0x00007FF7474AE17F]
	BaseThreadInitThunk [0x00007FF8231C257D+29]
	RtlUserThreadStart [0x00007FF82366AF28+40]


In [4]:
# import_to_csv(company_names,job_titles,posted_ons,job_details,image_src,type_location,locations,"Data")

In [5]:
# print(len(company_names))
# print(len(job_titles))
# print(len(job_details))
# print(len(posted_ons))
# print(len(image_src))
# print(len(type_location))
# print(len(locations))

In [6]:
# a = 0
# for id in job_ids :
#     job_click = driver.find_element(By.XPATH, "//a[@id='"+id+"']")
#     job_click.click()
#     time.sleep(1)
#     type_location = driver.find_element(By.XPATH, "//div[@class='js-match-insights-provider-g6kqeb ecydgvn0']")
#     print(type_location.text)
#     detail_find = driver.find_element(By.XPATH, "//div[@id='jobDescriptionText']")
#     print(detail_find.text)
#     img_find = driver.find_element(By.XPATH, "//div[@class='jobsearch-JobInfoHeader-title-container jobsearch-JobInfoHeader-title-containerEji css-82ysm6 eu4oa1w0']")
#     try:
#         img_element = img_find.find_element(By.TAG_NAME, "img")
#         img_element = img_element.get_attribute("src")
#         print(img_element)
#     except Exception as e :
#         print("")
#     print()
#     if a == 1 :
#         break

#     a += 1

In [7]:
# # Ambil semua nilai atribut "data-occludable-job-id" dari setiap <li>, dan hapus yang None
# job_ids = [li.get_attribute("data-occludable-job-id") for li in li_elements if li.get_attribute("data-occludable-job-id") is not None]
